In [ ]:
from google.colab import drive

drive.mount('/content/drive')
root_dir = "/content/drive/My Drive/"
wiki_dir = root_dir + 'Tesi/Wiki50/Wikipedia/'

In [ ]:
punctuations = ''''"“”’'<>–~'''

WIKI50

In [ ]:
base_dir = root_dir + 'Tesi/Wiki50/onlytxt/'
ann_dir = root_dir + 'Tesi/Wiki50/annotationWV/'

In [ ]:
from collections import defaultdict
from pathlib import Path
import pandas as pd

results = defaultdict(list)
for file in Path(base_dir).iterdir():
    with open(file, "r") as file_open:
        results["file_name"].append(file.name)
        results["text"].append(file_open.read())

In [ ]:
df = pd.DataFrame(results)

In [ ]:
import re

def remove_header(text):
  text = re.sub("\n", " ", text)
  return text

df['text'] = df['text'].apply(lambda x: remove_header(x))

In [ ]:
df = df.sort_values('file_name')

ANNOTATED TEXT

In [ ]:
results2 = defaultdict(list)
for file in Path(ann_dir).iterdir():
    with open(file, "r") as file_open:
        results2["file_name"].append(file.name)
        results2["annotation"].append(file_open.read())

In [ ]:
df2 = pd.DataFrame(results2)

In [ ]:
df2['annotation'] = df2['annotation'].apply(lambda x: remove_header(x))

In [ ]:
def remove_tab(text):
  text = re.sub("\t", " ", text)
  return text

df2['annotation'] = df2['annotation'].apply(lambda x: remove_tab(x))

In [ ]:
df2 = df2.sort_values('file_name')

CONCATENATION

In [ ]:
df = df.drop(columns=['file_name'])

In [ ]:
df2 = df2.drop(columns=['file_name'])

In [ ]:
df = df.reset_index()

df = df.drop(columns=['index'])

In [ ]:
df2 = df2.reset_index()

df2 = df2.drop(columns=['index'])

In [ ]:
wiki_df = df.join(df2.set_index(df.index[-len(df2):]))

In [ ]:
final_list = []

for index, row in wiki_df.iterrows():
    result = []
    index_list = re.findall(r'\d+ \d+', row['annotation'])
    for indexes in index_list:
        start, end = map(int, indexes.split())
        result.append(row['text'][start:end])
    final_list.append(result)

In [ ]:
wiki_df['result'] = final_list

In [ ]:
mwe_without_unigrams = []

In [ ]:
for i in range(len(wiki_df)):
  for kw in wiki_df['result'][i]:
    if(' ' in kw):
        kw = kw.lower()
        mwe_without_unigrams.append(kw)
  wiki_df['result'][i]=mwe_without_unigrams
  mwe_without_unigrams = []

In [ ]:
wiki_df = wiki_df.drop(columns=['annotation'])

In [ ]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in punctuations])
    return text

wiki_df['text'] = wiki_df['text'].apply(lambda x: remove_punct(x))

In [ ]:
nopunct_list=[]

In [ ]:
for i in range(len(wiki_df)):
  for el in wiki_df['result'][i]:
    el = el.translate(str.maketrans('', '', punctuations))
    nopunct_list.append(el)
  wiki_df["result"][i]=nopunct_list
  nopunct_list=[]

In [ ]:
wiki_df

SAVE

In [ ]:
compression_opts = dict(method='zip', archive_name='wiki50_preprocessed.csv')

wiki_df.to_csv('wiki50_preprocessed.zip', index=False, compression=compression_opts)